In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
with open("../json/response/nlp_response_m3.json", "r", encoding = "utf-8") as f:
    nlp_response = json.load(f)

In [3]:
bot_name = "m3SumOH__latest"
intent = "oh"
n_lookup = 20
result = {}

In [4]:
for call in nlp_response: # loop each call
    call_data = call["data"]
    call_name = call["fileName"]
    temp_result = {"start_index": None,
                   "entities": []}
    for i, data in enumerate(call_data): # loop each sentence in one call
        
        sum = data[bot_name]
        text = sum['text']
        intents = sum['intents']
        entities = sum['entities']
        entities_value = [x["entity"] for x in entities]
        
        if text != "":
            if intents[0]["label"] == intent:
                temp_result["start_index"] = i
                for j in range(n_lookup):
                    try:
                        entities_lookup = call_data[i + j][bot_name]["entities"]
                        lookup_value = [x["entity"] for x in entities_lookup]
                        if lookup_value[0] not in entities_value:
                            entities_value.extend(lookup_value)
                        continue
                    except:
                        break
            # if entities:
                temp_result["entities"] = entities_value
                break
    result[call_name] = temp_result

In [5]:
with open("../json/response/response_criteria/find_patterns.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [6]:
count = {}
start_index = []
for call_name in result:
    try:
        first_entity = result[call_name]["entities"][0]
    except:
        continue
    if first_entity not in count: count[first_entity] = 1
    else: count[first_entity] += 1
    start_index.append(result[call_name]["start_index"])

print(count)

{'oh_solution': 99, 'motivation_result': 3, 'motivation_title': 10, 'amount': 5, 'payment_datetime': 3}


In [7]:
start_index = np.array(start_index)
print(np.max(start_index))
print(np.min(start_index))
print(np.mean(start_index))
print(np.median(start_index))
print(np.quantile(start_index, 0.9))
print(np.quantile(start_index, 0.1))

41
4
11.633333333333333
9.0
23.0
4.9
